In [53]:
import pyodbc
import pandas as pd

In [54]:
pyodbc.drivers()

['PostgreSQL',
 'MySQL',
 'MySQL-5',
 'FreeTDS',
 'MariaDB',
 'ODBC Driver 18 for SQL Server',
 'ODBC Driver 17 for SQL Server']

In [55]:
conn = pyodbc.connect(
    Driver = '{ODBC Driver 18 for SQL Server}',
    Server = 'localhost',
    Database = 'DbFilme',
    Encrypt = 'Yes',
    TrustServerCertificate = 'Yes',
    UID = 'SA',
    PWD = 'Pg1024_JH', # Sua senha do Banco de Dados Local no SQL Server
    timeout=30
)

cursor = conn.cursor()

In [56]:
df = pd.read_csv("arquivos/consolidated/movies_consolidado.csv")

In [57]:
df.head()

,id,title,releaseDate,rating,genres,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,production_companies,director,writer,top_cast
0,2000556032,Aftersun,2022-10-21,R,Drama,Sophie reflects on the shared joy and private ...,98,Não informado,95,46,Universal acclaim,80,256,Generally favorable,"BBC Film,British Film Institute (BFI),Screen S...",Charlotte Wells,Charlotte Wells,"Paul Mescal,Frankie Corio,Celia Rowlson-Hall,S..."
1,2000556064,TÁR,2022-10-07,R,"Drama,Music",Set in the international world of classical mu...,158,Não informado,93,59,Universal acclaim,75,246,Generally favorable,"Focus Features,Standard Film Company,EMJAG Pro...",Todd Field,Todd Field,"Cate Blanchett,Noémie Merlant,Nina Hoss,Sophie..."
2,2000556607,No Bears,2022-12-23,Não informado,"Drama,Romance",Two parallel love stories in which the partner...,106,Não informado,93,29,Universal acclaim,80,23,Generally favorable,"JP Production,Janus Film,Janus Films,ARP Sélec...",Jafar Panahi,Jafar Panahi,"Jafar Panahi,Naser Hashemi,Vahid Mobasseri,Bak..."
3,2000556604,All the Beauty and the Bloodshed,2022-11-23,Unrated,"Documentary,History","All the Beauty and the Bloodshed is an epic, e...",113,Não informado,91,34,Universal acclaim,78,26,Generally favorable,"Participant,Praxis Films,I Wonder Pictures,Neo...",Laura Poitras,Não informado,"Nan Goldin,David Velasco,Megan Kapler,Marina B..."
4,2000554951,Hit the Road,2022-04-22,Não informado,"Comedy,Drama,Music",A family of four – two middle-aged parents and...,93,Não informado,91,28,Universal acclaim,72,27,Generally favorable,"Cherry Pickers Filmdistributie,Kino Lorber,Ang...",Panah Panahi,Panah Panahi,"Pantea Panahiha,Mohammad Hassan Madjooni,Rayan..."


In [58]:
df.columns

Index(['id', 'title', 'releaseDate', 'rating', 'genres', 'description',
       'duration', 'tagline', 'metascore', 'metascore_count',
       'metascore_sentiment', 'userscore', 'userscore_count',
       'userscore_sentiment', 'production_companies', 'director', 'writer',
       'top_cast'],
      dtype='object')

In [59]:
import time

block_size = 500 # quantidade de insert pra não explodir isso ai

# Monte de coisa pra driblar erro
def safe_int(value):
    try:
        if pd.isna(value):
            return None
        return int(value)
    except:
        return None

def safe_float(value):
    try:
        if pd.isna(value):
            return None
        return float(value)
    except:
        return None

def safe_str(value):
    if pd.isna(value):
        return None
    return str(value)

def safe_date(value):
    if pd.isna(value):
        return None
    if value == '0000-00-00':
        return None
    return value

In [60]:
c = 0

# A parte que insere os ngcs do filmes
for row in df.itertuples():
    try:
        cursor.execute('''
            INSERT INTO movies (
                movie_id, title, release_date, description,
                duration, tagline, metascore, metascore_count,
                userscore, userscore_count, userscore_sentiment,
                production_companies, director, writer, top_cast)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''',
            row.id,
            safe_str(row.title),
            safe_date(row.releaseDate),
            safe_str(row.description),
            safe_int(row.duration),
            safe_str(row.tagline),
            safe_int(row.metascore),
            safe_int(row.metascore_count),
            safe_int(row.userscore),
            safe_int(row.userscore_count),
            safe_str(row.userscore_sentiment),
            safe_str(row.production_companies),
            safe_str(row.director),
            safe_str(row.writer),
            safe_str(row.top_cast)
        )
        c += 1
        if c % block_size == 0:
            conn.commit()
            time.sleep(0.1) # pausa para aliviar essa bomba
    except Exception as e:
        print(f"Erro ao inserir linha: {row}\n→ {e}")

conn.commit()

Erro ao inserir linha: Pandas(Index=27, id=2000554696, title='Turning Red', releaseDate='2022-03-11', rating='TV-PG', genres='Animation,Adventure,Comedy,Drama,Family,Fantasy,Music,Romance', description='Mei Lee (voice of Rosalie Chiang) is a confident, dorky 13-year-old torn between staying her mother’s dutiful daughter and the chaos of adolescence. Her protective, if not slightly overbearing mother, Ming (voice of Sandra Oh), is never far from her daughter—an unfortunate reality for the teenager. And as if changes to her interests, relationships and body weren’t enough, whenever she gets too excited (which is practically ALWAYS), she “poofs” into a giant red panda!', duration=100, tagline='Growing Up is a Beast.', metascore=83, metascore_count=53, metascore_sentiment='Universal acclaim', userscore=62, userscore_count=556, userscore_sentiment='Generally favorable', production_companies='Walt Disney Pictures,Pixar Animation Studios,Québec Film & TV Production Tax Credit,HKC Entertainmen

OperationalError: ('08S01', '[08S01] [Microsoft][ODBC Driver 18 for SQL Server]Communication link failure (-2147467259) (SQLEndTran)')